In [1]:
# pip installs

!pip install -q --upgrade torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124 --index-url https://download.pytorch.org/whl/cu124
!pip install -q --upgrade requests==2.32.3 bitsandbytes==0.46.0 transformers==4.48.3 accelerate==1.3.0 datasets==3.2.0 peft==0.14.0 trl==0.14.0 matplotlib wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 101.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 78.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 33.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━

In [11]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from urllib.parse import urlparse
import logging
import optuna as optuna

from tqdm import tqdm

In [15]:
# ==============================
# Weights & Biases Login (Kaggle)
# ==============================

import os
import wandb
from kaggle_secrets import UserSecretsClient
# Project name
LOG_TO_WANDB = True   # or False

PROJECT_NAME = "Optuna-Tester"

# Get secret from Kaggle Secrets
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

# Strip whitespace just in case
wandb_api_key = wandb_api_key.strip()

# Login to W&B
wandb.login(key=wandb_api_key)

# Configure project settings
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: WARNING [wandb.login()] Changing session credentials to explicit value for https://api.wandb.ai.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import optuna

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
df = pd.read_csv('/kaggle/input/datasets/mdmarufmullah/traindata/data.csv')
x = df.iloc[:, :3].values  # First 3 columns as input features
y = df.iloc[:, 3].values.reshape(-1, 1)  # 4th column as target

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test  = torch.tensor(X_test,  dtype=torch.float32).to(device)

y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test  = torch.tensor(y_test,  dtype=torch.float32).to(device)

In [17]:
from dotenv import load_dotenv
import os

load_dotenv()  # loads .env automatically

False

In [18]:
def eval_metrics(actual, pred):
    mae = mean_absolute_error(actual, pred)
    mse = mean_squared_error(actual, pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [19]:
def objective(trial):
    import wandb

    # -------------------------------
    # Hyperparameters to tune
    # -------------------------------
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 5)
    neurons_per_layer = trial.suggest_int('neurons_per_layer', 8, 128, step=8)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    epochs = trial.suggest_int('epochs', 10, 50, step=10)

    # -------------------------------
    # Initialize W&B run (one per trial)
    # -------------------------------
    run = wandb.init(
        project=PROJECT_NAME,
        config={
            "num_hidden_layers": num_hidden_layers,
            "neurons_per_layer": neurons_per_layer,
            "dropout_rate": dropout_rate,
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "optimizer": optimizer_name,
            "weight_decay": weight_decay,
            "epochs": epochs
        },
        reinit=True
    )

    # -------------------------------
    # Build model
    # -------------------------------
    layers = []
    input_dim = X_train.shape[1]

    for _ in range(num_hidden_layers):
        layers.append(nn.Linear(input_dim, neurons_per_layer))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_rate))
        input_dim = neurons_per_layer

    layers.append(nn.Linear(input_dim, 1))
    model = nn.Sequential(*layers).to(device)

    # -------------------------------
    # Optimizer
    # -------------------------------
    if optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    else:
        optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    loss_fn = nn.MSELoss()

    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # -------------------------------
    # Training loop
    # -------------------------------
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0

        for xb, yb in train_loader:
            optimizer.zero_grad()
            loss = loss_fn(model(xb), yb)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        epoch_loss /= len(train_loader)

        # Log training loss per epoch
        wandb.log({"train_loss": epoch_loss, "epoch": epoch})

    # -------------------------------
    # Validation
    # -------------------------------
    model.eval()
    with torch.no_grad():
        val_loss = loss_fn(model(X_test), y_test).item()

    # Log validation loss
    wandb.log({"val_loss": val_loss})

    # Optional: log model
    wandb.watch(model)

    run.finish()

    return val_loss

In [20]:
import optuna

# Create Optuna study
study = optuna.create_study(direction="minimize")

# Run optimization
study.optimize(objective, n_trials=5)

# Print results
print("Best params:", study.best_params)
print("Best trial:")
print(study.best_trial.params)

[I 2026-02-15 18:50:34,219] A new study created in memory with name: no-name-5d1fa1bf-8f21-4968-a3bf-146ea3a3e679


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_loss,█▇▇▆▆▅▅▅▅▄▄▄▄▃▃▂▃▂▂▂▂▂▁▁▁▁▁▁▁▁
val_loss,▁
epoch,29
train_loss,0.45754
val_loss,0.2949


[I 2026-02-15 18:50:46,854] Trial 0 finished with value: 0.2949005961418152 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 8, 'dropout_rate': 0.4, 'learning_rate': 0.0024170652347462055, 'batch_size': 32, 'optimizer': 'SGD', 'weight_decay': 3.1564371911151945e-05, 'epochs': 30}. Best is trial 0 with value: 0.2949005961418152.


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train_loss,█▇▇▆▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁
val_loss,▁
epoch,19
train_loss,0.36757
val_loss,0.20201


...

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,██▇▇▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_loss,▁
epoch,39
train_loss,0.33614
val_loss,0.27908


[I 2026-02-15 18:51:15,669] Trial 3 finished with value: 0.2790786623954773 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 24, 'dropout_rate': 0.1, 'learning_rate': 0.0009068760807619664, 'batch_size': 64, 'optimizer': 'SGD', 'weight_decay': 1.533005437194489e-05, 'epochs': 40}. Best is trial 2 with value: 0.16043494641780853.


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train_loss,█▆▆▆▆▅▅▇▅▆▅▄▃▆▅▂▃▃▃▂▅▃▂▄▂▂▁▃▃▁
val_loss,▁
epoch,29
train_loss,0.83405
val_loss,0.70811


[I 2026-02-15 18:51:24,742] Trial 4 finished with value: 0.7081091403961182 and parameters: {'num_hidden_layers': 1, 'neurons_per_layer': 40, 'dropout_rate': 0.4, 'learning_rate': 5.446837752919621e-05, 'batch_size': 128, 'optimizer': 'RMSprop', 'weight_decay': 0.00015049678187865537, 'epochs': 30}. Best is trial 2 with value: 0.16043494641780853.


Best params: {'num_hidden_layers': 2, 'neurons_per_layer': 40, 'dropout_rate': 0.30000000000000004, 'learning_rate': 8.438309860732548e-05, 'batch_size': 16, 'optimizer': 'Adam', 'weight_decay': 0.0006706911176429714, 'epochs': 20}
Best trial:
{'num_hidden_layers': 2, 'neurons_per_layer': 40, 'dropout_rate': 0.30000000000000004, 'learning_rate': 8.438309860732548e-05, 'batch_size': 16, 'optimizer': 'Adam', 'weight_decay': 0.0006706911176429714, 'epochs': 20}


## For Optimizing the r2, rmse

In [7]:
def eval_metrics(actual, pred):
    mae = mean_absolute_error(actual, pred)
    mse = mean_squared_error(actual, pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [21]:
def objective(trial):
    import wandb

    # -------------------------------
    # Hyperparameters to tune
    # -------------------------------
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 5)
    neurons_per_layer = trial.suggest_int('neurons_per_layer', 8, 128, step=8)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5, step=0.1)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'SGD', 'RMSprop'])
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-3, log=True)
    epochs = trial.suggest_int('epochs', 10, 50, step=10)

    # -------------------------------
    # Initialize W&B run
    # -------------------------------
    run = wandb.init(
        project=PROJECT_NAME,
        group="Optuna-Study",
        config={
            "num_hidden_layers": num_hidden_layers,
            "neurons_per_layer": neurons_per_layer,
            "dropout_rate": dropout_rate,
            "learning_rate": learning_rate,
            "batch_size": batch_size,
            "optimizer": optimizer_name,
            "weight_decay": weight_decay,
            "epochs": epochs
        },
        reinit=True
    )

    # -------------------------------
    # Build model
    # -------------------------------
    layers = []
    input_dim = X_train.shape[1]

    for _ in range(num_hidden_layers):
        layers.append(nn.Linear(input_dim, neurons_per_layer))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout_rate))
        input_dim = neurons_per_layer

    layers.append(nn.Linear(input_dim, 1))
    model = nn.Sequential(*layers).to(device)

    # -------------------------------
    # Optimizer
    # -------------------------------
    if optimizer_name == 'Adam':
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_name == 'SGD':
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    else:
        optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    loss_fn = nn.MSELoss()

    train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # -------------------------------
    # Training loop
    # -------------------------------
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0

        for xb, yb in train_loader:
            optimizer.zero_grad()
            loss = loss_fn(model(xb), yb)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        epoch_loss /= len(train_loader)

        # Log training loss
        wandb.log({"train_loss": epoch_loss, "epoch": epoch})

    # -------------------------------
    # Validation
    # -------------------------------
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test)
        val_loss = loss_fn(y_pred, y_test).item()

        y_true = y_test.cpu().numpy()
        y_pred = y_pred.cpu().numpy()

        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

    # Log metrics to W&B
    wandb.log({
        "val_loss": val_loss,
        "rmse": rmse,
        "mae": mae,
        "r2": r2
    })

    # Optional: watch gradients
    wandb.watch(model)

    run.finish()

    # IMPORTANT: Optuna must return a single value
    return val_loss, rmse, mae, r2


In [22]:
# Create Optuna study

study = optuna.create_study(directions=["minimize","minimize","minimize", "maximize"])
study.optimize(objective, n_trials=5)

[I 2026-02-15 18:56:33,130] A new study created in memory with name: no-name-13e4017a-d149-4057-b71a-8dc9e1cde696


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
mae,▁
r2,▁
rmse,▁
train_loss,█▄▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁
epoch,19
mae,0.31188
r2,0.83264
rmse,0.38411
train_loss,0.20236


[I 2026-02-15 18:56:43,016] Trial 0 finished with values: [0.14753875136375427, 0.3841077146616468, 0.3118775486946106, 0.832642138004303] and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 64, 'dropout_rate': 0.30000000000000004, 'learning_rate': 0.001391267294048847, 'batch_size': 64, 'optimizer': 'Adam', 'weight_decay': 6.284665058125246e-05, 'epochs': 20}.


epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
mae,▁
r2,▁
rmse,▁
train_loss,█▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,▁
epoch,29
mae,0.29111
r2,0.84074
rmse,0.3747
train_loss,0.16553


[I 2026-02-15 18:56:53,138] Trial 1 finished with values: [0.14039896428585052, 0.3746984978430665, 0.29110559821128845, 0.8407410383224487] and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 56, 'dropout_rate': 0.2, 'learning_rate': 0.0012853395088719378, 'batch_size': 32, 'optimizer': 'Adam', 'weight_decay': 1.657402380459818e-05, 'epochs': 30}.


...

[I 2026-02-15 18:57:03,802] Trial 2 finished with values: [0.15633180737495422, 0.3953881730337343, 0.3124212920665741, 0.8226678967475891] and parameters: {'num_hidden_layers': 4, 'neurons_per_layer': 88, 'dropout_rate': 0.30000000000000004, 'learning_rate': 0.004440006614695958, 'batch_size': 32, 'optimizer': 'SGD', 'weight_decay': 0.0008512984172298284, 'epochs': 50}.


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
mae,▁
r2,▁
rmse,▁
train_loss,█▂▂▂▂▂▂▂▂▂▁▁▁▁▂▂▁▁▂▂▂▁▁▁▁▁▂▂▁▁▁▁▁▁▁▂▁▁▁▁
val_loss,▁
epoch,39
mae,0.30751
r2,0.83272
rmse,0.38402
train_loss,0.17406


[I 2026-02-15 18:57:15,191] Trial 3 finished with values: [0.1474713385105133, 0.38401997149954753, 0.3075103759765625, 0.8327186107635498] and parameters: {'num_hidden_layers': 5, 'neurons_per_layer': 112, 'dropout_rate': 0.2, 'learning_rate': 0.0011665830581149932, 'batch_size': 32, 'optimizer': 'RMSprop', 'weight_decay': 1.889725182014699e-05, 'epochs': 40}.


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
mae,▁
r2,▁
rmse,▁
train_loss,███▇▇▇▇▆▆▆▅▅▄▄▃▃▃▂▂▁
val_loss,▁
epoch,19
mae,0.55057
r2,0.50554
rmse,0.66023
train_loss,0.54764


[I 2026-02-15 18:57:24,430] Trial 4 finished with values: [0.435907244682312, 0.6602327426632478, 0.5505741238594055, 0.5055366158485413] and parameters: {'num_hidden_layers': 3, 'neurons_per_layer': 80, 'dropout_rate': 0.1, 'learning_rate': 0.002626685099237075, 'batch_size': 64, 'optimizer': 'SGD', 'weight_decay': 1.2421461263780838e-05, 'epochs': 20}.


In [ ]:
print(f"Number of Pareto-optimal trials: {len(study.best_trials)}")
# here multiple varialbles are maximized or minimized
for t in study.best_trials:
    print("Trial#", t.number)
    print("  Params:", t.params)
    print("  Values (RMSE, MAE, R2):", t.values)
    print()


Number of Pareto-optimal trials: 1
Trial# 4
  Params: {'num_hidden_layers': 3, 'neurons_per_layer': 48, 'dropout_rate': 0.2, 'learning_rate': 0.002847906586326214, 'batch_size': 32, 'optimizer': 'Adam', 'weight_decay': 1.0362562619071438e-05, 'epochs': 10}
  Values (RMSE, MAE, R2): [0.14456802606582642, 0.38022100925445557, 0.30004870891571045, 0.857831597328186]

